In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [2]:
import pandas as pd
import pandas_profiling as pds
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("../data/raw/exoplanet_data.csv")
# Drop the null columns where all values are null
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6991 entries, 0 to 6990
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    6991 non-null   object 
 1   koi_fpflag_nt      6991 non-null   int64  
 2   koi_fpflag_ss      6991 non-null   int64  
 3   koi_fpflag_co      6991 non-null   int64  
 4   koi_fpflag_ec      6991 non-null   int64  
 5   koi_period         6991 non-null   float64
 6   koi_period_err1    6991 non-null   float64
 7   koi_period_err2    6991 non-null   float64
 8   koi_time0bk        6991 non-null   float64
 9   koi_time0bk_err1   6991 non-null   float64
 10  koi_time0bk_err2   6991 non-null   float64
 11  koi_impact         6991 non-null   float64
 12  koi_impact_err1    6991 non-null   float64
 13  koi_impact_err2    6991 non-null   float64
 14  koi_duration       6991 non-null   float64
 15  koi_duration_err1  6991 non-null   float64
 16  koi_duration_err2  6991 

In [6]:

# Select your features (columns)

# Set features. This will also be used as your x values.
# selected_features = df[['names', 'of', 'selected', 'features', 'here']]

In [7]:
selected_features = df[["koi_disposition", "koi_fpflag_nt", "koi_fpflag_ss", "koi_fpflag_co", "koi_fpflag_ec", "koi_period", "koi_time0bk", "koi_impact", "koi_duration", "koi_depth", "koi_prad", "koi_teq","koi_insol","koi_model_snr", "koi_tce_plnt_num", "koi_steff", "koi_slogg", "koi_srad", "ra", "dec", "koi_kepmag"]]
selected_features.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,...,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,...,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,...,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,...,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,...,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,...,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714


In [8]:
confirmed_features = selected_features.loc[selected_features["koi_disposition"]=="CANDIDATE"]
confirmed_features.tail()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,...,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6979,CANDIDATE,0,0,0,0,4.736816,131.787600,0.218,2.8400,35.3,...,1137,395.05,6.9,1,6088,4.456,1.011,289.20331,44.505138,13.922
6980,CANDIDATE,0,0,0,0,130.235324,218.271900,0.075,5.6780,750.1,...,332,2.86,9.7,1,5616,4.529,0.903,289.57452,44.519939,15.991
6984,CANDIDATE,0,0,0,0,8.870416,137.481093,1.206,1.2864,873.1,...,1151,414.26,43.8,1,6022,4.027,1.514,290.14914,50.239178,13.579
6985,CANDIDATE,0,0,0,0,47.109631,144.131720,1.230,5.7410,752.2,...,751,75.40,35.1,1,5258,3.597,2.780,296.15601,44.920090,13.731
6988,CANDIDATE,0,0,0,0,1.739849,133.001270,0.043,3.1140,48.5,...,1608,1585.81,10.6,1,6119,4.444,1.031,286.50937,47.163219,14.757


# Create a Train Test Split

Use `koi_disposition` for the y values

In [9]:
data = selected_features.values
X = data[:, 1:21]
y = selected_features["koi_disposition"].values.reshape(-1, 1)
print(X.shape, y.shape)

(6991, 20) (6991, 1)


In [10]:
from sklearn.preprocessing import LabelEncoder
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
encoded_y

C:\Users\Donley\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


array([1, 2, 2, ..., 0, 2, 2])

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, random_state=1)

In [12]:
# X_train.head()
# for label, original_class in zip(encoded_y, y):
#     print('Original Class: ' + str(original_class))
#     print('Encoded Label: ' + str(label))
#     print('-' * 12)

In [13]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y.shape

(6991, 3)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [14]:
from sklearn.preprocessing import MinMaxScaler
# Scale your data
# build the scaler model
scaler = MinMaxScaler()
# fit using the train set
scaler.fit(X)
# transform the test test
X_scaled = scaler.transform(X)
# Verify minimum value of all features
X_scaled.min(axis=0)

In [14]:
 X_test_scaled

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [15]:
# Verify maximum value of all features
X_scaled.max(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

In [16]:
# Manually normalise without using scikit-learn
X_manual_scaled = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
# Verify manually VS scikit-learn estimation
# print(np.allclose(X_scaled, X_manual_scaled))
np.isfinite(X_scaled, X_manual_scaled)
# True

array([[True, True, True, ..., True, True, True],
       [True, True, True, ..., True, True, True],
       [True, True, True, ..., True, True, True],
       ...,
       [True, True, True, ..., True, True, True],
       [True, True, True, ..., True, True, True],
       [True, True, True, ..., True, True, True]], dtype=object)

In [17]:
# One-hot encoding
# y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [18]:
# first, create a normal neural network with 3 inputs, 20 hidden nodes, and 3 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=20, activation='relu', input_dim=20))
model.add(Dense(units=3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                420       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 63        
Total params: 483
Trainable params: 483
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the Model



In [20]:
# Fit the model to the training data
model.fit(
    X_scaled,
    one_hot_y,
    epochs=500,
    shuffle=True,
    verbose=2
)

Epoch 1/500
219/219 - 0s - loss: 0.9102 - accuracy: 0.4596
Epoch 2/500
219/219 - 0s - loss: 0.5484 - accuracy: 0.7724
Epoch 3/500
219/219 - 0s - loss: 0.4297 - accuracy: 0.7937
Epoch 4/500
219/219 - 0s - loss: 0.3960 - accuracy: 0.7923
Epoch 5/500
219/219 - 0s - loss: 0.3831 - accuracy: 0.7956
Epoch 6/500
219/219 - 0s - loss: 0.3766 - accuracy: 0.7969
Epoch 7/500
219/219 - 0s - loss: 0.3730 - accuracy: 0.7965
Epoch 8/500
219/219 - 0s - loss: 0.3695 - accuracy: 0.8003
Epoch 9/500
219/219 - 0s - loss: 0.3683 - accuracy: 0.8000
Epoch 10/500
219/219 - 0s - loss: 0.3662 - accuracy: 0.8006
Epoch 11/500
219/219 - 0s - loss: 0.3651 - accuracy: 0.8022
Epoch 12/500
219/219 - 0s - loss: 0.3636 - accuracy: 0.8059
Epoch 13/500
219/219 - 0s - loss: 0.3631 - accuracy: 0.8040
Epoch 14/500
219/219 - 0s - loss: 0.3614 - accuracy: 0.8083
Epoch 15/500
219/219 - 0s - loss: 0.3614 - accuracy: 0.8089
Epoch 16/500
219/219 - 0s - loss: 0.3600 - accuracy: 0.8063
Epoch 17/500
219/219 - 0s - loss: 0.3601 - accura

In [31]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

ValueError: Data cardinality is ambiguous:
  x sizes: 6991
  y sizes: 1748
Please provide data which shares the same first dimension.

In [30]:
import matplotlib.pyplot as plt

# plt.plot(range((model_loss*100)), (model_accuracy*100), 
#          label='Training')
# plt.plot(range((model_loss*100)), (model_accuracy*100), 
#          label='Validation', linestyle='--')
# plt.ylabel('Accuracy')
# plt.xlabel('Epochs')
# plt.legend(loc='lower right')
# plt.savefig('01.png', dpi=300)
# plt.show()

TypeError: 'float' object cannot be interpreted as an integer

In [24]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

AttributeError: 'Sequential' object has no attribute 'score'

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)